## SegVol inference

In [ ]:
from adapt_med_seg.pipelines.evaluate import EvaluateArgs, EvaluatePipeline

In [ ]:

model_names = ["segvol_baseline"]
dataset_paths = ["../datasets/chaos_000"] # assuming notebooks/preprocess_data.ipynb has been run 
modality_lists = [["CT"], ["MR"]]

for model_name in model_names:
    for modalities in modality_lists:
        for dataset_path in dataset_paths:
            eval_pipeline = EvaluatePipeline(
                evaluate_args=EvaluateArgs(
                    dataset_path=dataset_path,
                    model_name=model_name,
                    cls_idx=0,
                    device='cuda',
                    modalities=modalities, # we look at each modality separately
                    # use_wandb=True,
                )
            )

            results = eval_pipeline.run()
            print(f'Results for {model_name} and modality {modalities[0]} on dataset {dataset_path}:\n{results}')